# **Create Tables (Standings) Data**

##### Imports

In [1]:
import numpy as np
import pandas as pd
import re
from tqdm.notebook import tqdm

### Standings and Result functions

In [2]:
def standings(frame, result_col, goals_col, goals_opp_col, points_col):
    """This function takes in a DataFrame and strings identifying fields to calculate the league table.
    (Making it generalized will allow us to calculate league tables for First Half Goals only. Second Half Goals only.)

    frame: DataFrame containing the match data
    result_col: column indicating the match result
    goals_col: column of the Home team goals
    goals_opp_col: column of the Away team goals
    points_col: column of the points associated with the result
    """
    record = {}
    record['Played'] = np.size(frame[result_col])
    record['Won'] = np.sum(frame[result_col] == 'W')
    record['Drawn'] = np.sum(frame[result_col] == 'D')
    record['Lost'] = np.sum(frame[result_col] == 'L')
    record['GF'] = np.sum(frame[goals_col])
    record['GA'] = np.sum(frame[goals_opp_col])
    record['GD'] = record['GF'] - record['GA']
    record['Points'] = np.sum(frame[points_col])
    
    return pd.Series(record,index=['Played', 'Won', 'Drawn', 'Lost', 'GF', 'GA', 'GD', 'Points'])

In [3]:
def get_result(score, score_opp):
    """Considering the relation between Home and Away team's goals, this functions returns the result of the considered match (Draw, Win, or Loss).
    
    score: goals scored by the Home team
    score_opp: goals scored by the Away team
    """
    if score == score_opp:
        return 'D'
    elif score > score_opp:
        return 'W'
    else:
        return 'L'

### Create Table function

In [4]:
def create_table(results_df, type):
    """This function takes in a DataFrame and outputs a datframe containing the standings for each season and round. 
     
    results_df: DataFrame containing the match data
    type: provides the possibility to specify if the desired table considers all games ('full'), only home games ('home'), or only away games ('away'). In case of a incorrect specification of such arguement the function will print a warning
    """
    
    # If type arguement is incorrectly specified, print a warning and stops the function 
    if type not in ['full', 'home', 'away']:
        return print("type arguement incorrectly specified!\nChoose between 'full', 'home', or 'away'")

    # Rename DataFrame columns
    results_df.rename(columns={'league_id':'Div','time_starting_at_date_time':'Date','scores_home_score':'FTHG','scores_away_score':'FTAG','result':'FTR', 'home_name':'HomeTeam', 'away_name':'AwayTeam'}, inplace=True) 
    # Change Date format 
    results_df['Date'] = pd.to_datetime(results_df['Date'], format = '%Y-%m-%d').dt.date
    # Convert results from integer to string
    results_df['FTR'] = np.where(results_df['FTR'] == 0, 'D', np.where(results_df['FTR'] == 1, 'H', 'A'))
    # Obtain Half-time scores for both Home and Away team
    results_df['HTHG'] = [int(re.search('(\d+)-(\d+)', values).group(1)) for values in results_df['scores_ht_score']]
    results_df['HTAG'] = [int(re.search('(\d+)-(\d+)', values).group(2)) for values in results_df['scores_ht_score']]
    # Get (string) results at Half-time
    results_df['HTR'] = np.where(results_df['HTHG'] > results_df['HTAG'], 'H', np.where(results_df['HTHG'] == results_df['HTAG'], 'D', 'A'))
    # Drop not useful columns
    results_df.drop(['league_name', 'season_name', 'scores_ht_score'], axis=1, inplace=True)
    # Create a copy of existing columns for HomeTeam and AwayTeam
    results_df['H'] = results_df['HomeTeam']
    results_df['A'] = results_df['AwayTeam']
    # List of columns to keep 
    cols_to_keep = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'id', 'round_name', 'season_id']
    
    # Empty list to store data
    appended_data = []

    for i in tqdm(results_df['season_id'].unique()): # loop for each unique season     
        for j in results_df['round_name'].unique(): # loop for each unique round in the season 
            # Consider only results from i-th season and j-th round 
            res_byseasonround = results_df[(results_df['season_id'] == i) & (results_df['round_name'] <= j)] 
            # Unpivot a DataFrame from wide to long format
            team_res = pd.melt(res_byseasonround, id_vars=cols_to_keep, value_vars=['H', 'A'], var_name='Home/Away', value_name='Team') 
            team_res['Opponent'] = np.where(team_res['Team'] == team_res['HomeTeam'], team_res['AwayTeam'], team_res['HomeTeam'])
            points_map = {'W': 3,'D': 1,'L': 0} # associates point to results 
            # Full time goals, resuls and points
            team_res['Goals'] = np.where(team_res['Team'] == team_res['HomeTeam'],team_res['FTHG'],team_res['FTAG']) 
            team_res['Goals_Opp'] = np.where(team_res['Team'] != team_res['HomeTeam'],team_res['FTHG'],team_res['FTAG']) 
            team_res['Result'] = np.vectorize(get_result)(team_res['Goals'], team_res['Goals_Opp'])
            team_res['Points'] = team_res['Result'].map(points_map)
            # 1st half goals, resuls and points
            team_res['1H_Goals'] = np.where(team_res['Team'] == team_res['HomeTeam'],team_res['HTHG'],team_res['HTAG'])
            team_res['1H_Goals_Opp'] = np.where(team_res['Team'] != team_res['HomeTeam'],team_res['HTHG'],team_res['HTAG'])
            team_res['1H_Result'] = np.vectorize(get_result)(team_res['1H_Goals'], team_res['1H_Goals_Opp'])
            team_res['1H_Points'] = team_res['1H_Result'].map(points_map)
            # 2nd half goals, resuls and points
            team_res['2H_Goals'] = team_res['Goals'] - team_res['1H_Goals']
            team_res['2H_Goals_Opp'] = team_res['Goals_Opp'] - team_res['1H_Goals_Opp']
            team_res['2H_Result'] = np.vectorize(get_result)(team_res['2H_Goals'], team_res['2H_Goals_Opp'])
            team_res['2H_Points'] = team_res['2H_Result'].map(points_map)
            # Drop unnecessary columns and sort by date
            cols_to_drop = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']
            
            # To produce the specified table (either full, only home, or only away): filters (or not) the data depending on the choice
            if type == 'full':
                pass
            elif type == 'home':
                team_res = team_res[team_res['Home/Away'] == 'H']
            elif type == 'away':
                team_res = team_res[team_res['Home/Away'] == 'A']
            # Sort the DataFrame values by Date and id + Group them by team
            team_res = (team_res.sort_values(by=['Date', 'id']))
            res_byteam = team_res.groupby(['Team'])
            
            # Rank Teams in Standings
            table_FT = (res_byteam.apply(standings,result_col='Result',goals_col='Goals',goals_opp_col='Goals_Opp',points_col='Points').sort_values(by=['Points', 'GD', 'GF'], ascending=False))
            table_FT['rank'] = (table_FT.apply(lambda row: (row['Points'], row['GD'], row['GF']), axis=1).rank(method='min', ascending=False).astype(int)) 
            table_FT['Type'] = 'FT' 
            # Get League table_1H for First Half Goals only
            table_1H = (res_byteam.apply(standings,result_col='1H_Result',goals_col='1H_Goals',goals_opp_col='1H_Goals_Opp',points_col='1H_Points').sort_values(by=['Points', 'GD', 'GF'], ascending=False))
            table_1H['rank'] = (table_1H.apply(lambda row: (row['Points'], row['GD'], row['GF']), axis=1).rank(method='min', ascending=False).astype(int)) 
            table_1H['Type'] = '1H'
            # Get League Table_2H for First Half Goals only
            table_2H = (res_byteam.apply(standings,result_col='2H_Result',goals_col='2H_Goals',goals_opp_col='2H_Goals_Opp',points_col='2H_Points').sort_values(by=['Points', 'GD', 'GF'], ascending=False)) 
            table_2H['rank'] = (table_2H.apply(lambda row: (row['Points'], row['GD'], row['GF']), axis=1).rank(method='min', ascending=False).astype(int))
            table_2H['Type'] = '2H'

            # Concatenate the three tables (Full-time, 1st-half, and 2nd-half)
            table_merged = pd.concat([table_FT,table_1H, table_2H]) 
            # Add round and season columns
            table_merged['round_name'] = j 
            table_merged['season_id'] = i
            # Append observations to appended_data list 
            appended_data.append(table_merged)  
    
    # See pd.concat documentation for more info
    appended_data = pd.concat(appended_data)
    return appended_data

## Create and Store Tables Data

##### Full Table

In [5]:
needed_cols = ['id', 'league_id', 'season_id', 'round_name', 'time_starting_at_date_time', 'home_name', 'away_name', 'scores_home_score', 'scores_away_score', 'scores_ht_score', 'result', 'league_name', 'season_name']

In [6]:
results = pd.read_csv('../../Data/From_Preparation/match_cleaned.csv', low_memory=False)
results = results.loc[results['league_is_cup'] == 0, needed_cols]
full_table = create_table(results_df=results, type='full')
full_table.to_csv('../../Data/From_Collection/Standings_output/tables_FULL.csv')
full_table.tail()

  0%|          | 0/35 [00:00<?, ?it/s]

,Played,Won,Drawn,Lost,GF,GA,GD,Points,rank,Type,round_name,season_id
Team,,,,,,,,,,,,
Cagliari,32,8,9,15,21,35,-14,33,16,2H,38.0,18576
Spezia,33,7,9,17,21,33,-12,30,17,2H,38.0,18576
Venezia,31,7,8,16,13,34,-21,29,18,2H,38.0,18576
Genoa,33,4,16,13,18,30,-12,28,19,2H,38.0,18576
Salernitana,30,6,9,15,13,34,-21,27,20,2H,38.0,18576


##### Home Table

In [7]:
results = pd.read_csv('../../Data/From_Preparation/match_cleaned.csv', low_memory=False)
results = results.loc[results['league_is_cup'] == 0, needed_cols]
home_table = create_table(results_df=results, type='home')
home_table.to_csv('../../Data/From_Collection/Standings_output/tables_HOME.csv')
home_table.tail()

  0%|          | 0/35 [00:00<?, ?it/s]

,Played,Won,Drawn,Lost,GF,GA,GD,Points,rank,Type,round_name,season_id
Team,,,,,,,,,,,,
Spezia,16,5,3,8,15,16,-1,18,16,2H,38.0,18576
Torino,16,3,7,6,12,12,0,16,17,2H,38.0,18576
Cagliari,16,4,4,8,12,18,-6,16,18,2H,38.0,18576
Genoa,16,2,9,5,8,12,-4,15,19,2H,38.0,18576
Venezia,16,3,6,7,7,17,-10,15,20,2H,38.0,18576


##### Away Table

In [8]:
results = pd.read_csv('../../Data/From_Preparation/match_cleaned.csv', low_memory=False)
results = results.loc[results['league_is_cup'] == 0, needed_cols]
away_table = create_table(results_df=results, type='away')
away_table.to_csv('../../Data/From_Collection/Standings_output/tables_AWAY.csv')
away_table.tail()

  0%|          | 0/35 [00:00<?, ?it/s]

,Played,Won,Drawn,Lost,GF,GA,GD,Points,rank,Type,round_name,season_id
Team,,,,,,,,,,,,
Venezia,15,4,2,9,6,17,-11,14,16,2H,38.0,18576
Bologna,15,3,4,8,9,16,-7,13,17,2H,38.0,18576
Genoa,17,2,7,8,10,18,-8,13,18,2H,38.0,18576
Spezia,17,2,6,9,6,17,-11,12,19,2H,38.0,18576
Salernitana,15,1,5,9,6,22,-16,8,20,2H,38.0,18576
